## 워드클라우드를 위한 text_dictionary 제작

In [13]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os 

# 임의 데이터 형식 통일 
# 하나의 파일로 데이터 다 통합하기

"""뉴스 영상 스크래핑 구간 
위기발생 - 1달 ~ 위기 대응 + 1달

유튜브 영상 스크래핑 구간
1. 위기발생 전 2년 ~ 위기 발생 전날
2. 위기 발생 당일 ~ 위기 대응 전날
3. 위기 대응 날 ~ 1달"""

information = [["아이린","2020.10.20","2020.10.22"],["조현아","2014.12.05","2014.12.08"],["박나래","2021.03.24","2021.03.25"],["설현","2016.05.03","2016.05.12"],
               ["양팡","2020.08.05","2020.08.08"],["강민경","2020.07.15","2020.07.17"],["유희열","2022.06.14","2022.06.14"],
               ["홍진영","2020.11.05","2020.11.06"],["설민석","2020.12.29","2020.12.29"]]

for i in range(len(information)):
    information[i][1] = pd.to_datetime(information[i][1]).date()
    information[i][2] = pd.to_datetime(information[i][2]).date()



In [14]:
# 파일 df으로 read 시켜주기

naver = pd.read_csv("naver.csv")
naver_divided_comment = pd.read_csv("naver_divided_comment.csv")
yt_transcript = pd.read_csv("yt_transcript.csv")
yt_comment = pd.read_csv("yt_comment.csv")

In [15]:
# video_id로 댓글에 published date 달아주기
yt_comment = yt_comment.merge(yt_transcript.loc[:,["Video_ID","Publish Date"]], on="Video_ID",how="inner")

# df내 object들을 datetime object로 바꿔주기
naver["Date"] = pd.to_datetime(naver["Date"]).dt.date
yt_transcript["Publish Date"] = pd.to_datetime(yt_transcript["Publish Date"]).dt.date
yt_comment["Publish Date"] = pd.to_datetime(yt_comment["Publish Date"]).dt.date

In [16]:
import re

# 기간별로 text 나눠서 text 분석을 할 수 있도록 dictionary꼴로 만들어줌!
text_dictionary = {}

for info in information:
    query = info[0]

    naver_comment1 = naver.loc[(naver["query"]==query) & (naver["Date"] < info[1]) , "comments"]
    naver_comment1 = " ".join(naver_comment1.dropna().astype(str))

    naver_comment2 = naver.loc[(naver["query"]==query) & (naver["Date"] >= info[1]) & (naver["Date"] < info[2]), "comments"]
    naver_comment2 = " ".join(naver_comment2.dropna().astype(str))

    naver_comment3 = naver.loc[(naver["query"]==query) & (naver["Date"] >= info[2]), "comments"]
    naver_comment3 = " ".join(naver_comment3.dropna().astype(str))
    

    naver_content1 = naver.loc[(naver["query"]==query) & (naver["Date"] < info[1]) , "Content"]
    naver_content1 = " ".join(naver_content1.dropna().astype(str))

    naver_content2 = naver.loc[(naver["query"]==query) & (naver["Date"] >= info[1]) & (naver["Date"] < info[2]), "Content"]
    naver_content2 = " ".join(naver_content2.dropna().astype(str))

    naver_content3 = naver.loc[(naver["query"]==query) & (naver["Date"] >= info[2]), "Content"]
    naver_content3 = " ".join(naver_content3.dropna().astype(str))

    
    youtube_transcript1 = yt_transcript.loc[(yt_transcript["query"]==query) & (yt_transcript["Publish Date"] < info[1]) , "Transcript"]
    youtube_transcript1 = " ".join(youtube_transcript1.dropna().astype(str))

    youtube_transcript2 = yt_transcript.loc[(yt_transcript["query"]==query) & (yt_transcript["Publish Date"] >= info[1]) & (yt_transcript["Publish Date"] < info[2]) , "Transcript"]
    youtube_transcript2 = " ".join(youtube_transcript2.dropna().astype(str))
    
    youtube_transcript3 = yt_transcript.loc[(yt_transcript["query"]==query) & (yt_transcript["Publish Date"] >= info[2]) , "Transcript"]
    youtube_transcript3 = " ".join(youtube_transcript3.dropna().astype(str))

    youtube_comment1 = yt_comment.loc[(yt_comment["query"]==query) & (yt_comment["Publish Date"] < info[1]) , "comment_text"]
    youtube_comment1 = " ".join(youtube_comment1.dropna().astype(str))

    youtube_comment2 = yt_comment.loc[(yt_comment["query"]==query) & (yt_comment["Publish Date"] >= info[1]) & (yt_comment["Publish Date"] < info[2]) , "comment_text"]
    youtube_comment2 = " ".join(youtube_comment2.dropna().astype(str))
    
    youtube_comment3 = yt_comment.loc[(yt_comment["query"]==query) & (yt_comment["Publish Date"] >= info[2]) , "comment_text"]
    youtube_comment3 = " ".join(youtube_comment3.dropna().astype(str))

    new_dict = {
                "naver_comment1" : naver_comment1,"naver_comment2" : naver_comment2,"naver_comment3" : naver_comment3,
                "naver_content1" : naver_content1,"naver_content2" : naver_content2,"naver_content3" : naver_content3,
                "youtube_transcipt1" : youtube_transcript1,"youtube_transcipt2" : youtube_transcript2,"youtube_transcipt3" : youtube_transcript3, 
                "youtube_comment1" : youtube_comment1,"youtube_comment2" : youtube_comment2,"youtube_comment3" : youtube_comment3
                }
    text_dictionary[query] = new_dict

# 워드클라우드 만들기

In [17]:
# naver1, 2, 3 ....형태가 아니라 인물별로 시기1 시기2 시기3로 만들기

def update_text_dictionary(text_dictionary):
    for query, content_dict in text_dictionary.items():
        text_dictionary[query] = {
            "시기1": ' '.join([content_dict.get(f'{source}1', '') for source in ['naver_comment', 'naver_content', 'youtube_transcript', 'youtube_comment']]),
            "시기2": ' '.join([content_dict.get(f'{source}2', '') for source in ['naver_comment', 'naver_content', 'youtube_transcript', 'youtube_comment']]),
            "시기3": ' '.join([content_dict.get(f'{source}3', '') for source in ['naver_comment', 'naver_content', 'youtube_transcript', 'youtube_comment']]),
        }

update_text_dictionary(text_dictionary)

In [18]:
from kiwipiepy import Kiwi
import pandas as pd
from kiwipiepy.utils import Stopwords
import re

kiwi = Kiwi()
stopwords = Stopwords()

# 전처리 코드
def do_Kr_preprocessing(text, include_tags=['NN', 'NNG', 'NNP', 'MAG',"VA"]):
    filtered_content = re.sub('[^\s\w\d]', ' ', text)
    kiwi_tokens = kiwi.tokenize(filtered_content, stopwords=stopwords)
    selected_words = [token.form for token in kiwi_tokens if token.tag in include_tags and len(token.form) > 1 and len(token.form) < 180]
    return selected_words


from collections import Counter

# 빈도 분석!

columns = ['query', 'key', 'count']
freq_df = pd.DataFrame(columns=columns)


for info in information:
    query = info[0]
    for key in list(text_dictionary[query].keys()):
        conversion = text_dictionary[query][key]
        final_nouns = do_Kr_preprocessing(conversion)
        c = Counter(final_nouns)
        adding_df = pd.DataFrame({"query" : [query], "key" : [key], "count" : [str(c.most_common(30))]})
        freq_df = pd.concat([freq_df,adding_df], ignore_index=True)

freq_df.to_csv("frequency_analysis2.csv",index=False)


<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/var/folders/m5/bfzh8qks2fx0rrw3lkq1q4s80000gn/T/ipykernel_58418/1368753592.py:11: SyntaxWarning: invalid escape sequence '\s'
  filtered_content = re.sub('[^\s\w\d]', ' ', text)


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from wordcloud import WordCloud, STOPWORDS
import ast

plt.rcParams['font.family'] = 'AppleGothic'  # 한글 폰트 설정
file_path = 'frequency_analysis2.csv'
sent_df = pd.read_csv(file_path)

# 문자열 리스트를 딕셔너리로 변환
sent_df['count'] = sent_df['count'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else [])

def soft_color_func(*args, **kwargs):
    return "hsl(%d, 75%%, 50%%)" % np.random.choice(range(0, 200))  # HSL 색상 중 파스텔 블루-그린

def plot_wordcloud(pdf):
    for idx, row in sent_df.iterrows():
        if row['count']:  # 값이 있는 경우만 시각화
            word_freq = dict(row['count'])
            wordcloud = WordCloud(
                font_path='/Library/Fonts/AppleGothic.ttf',
                background_color='white',
                width=500,  # 정사각형 비율
                height=500,
                max_words=200,
                color_func=soft_color_func,  # 색상 함수 적용
                contour_color='gray',
                contour_width=1
            ).generate_from_frequencies(word_freq)
            
            plt.figure(figsize=(6, 6))  # 정사각형 플롯
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.title(f"Query: {row['query']} | Key: {row['key']}", fontsize=14, color='black')
            plt.axis('off')  # 축 제거
            plt.tight_layout()
            pdf.savefig()
            plt.close()

def save_plots_to_pdf(file_path, plot_function):
    with PdfPages(file_path) as pdf:
        plot_function(pdf)

save_plots_to_pdf('frequency_analysis_wordcloud_report4.pdf', plot_wordcloud)

# 추세선 제작

In [ ]:
# sentiment들 일일이 라벨링하고 라벨링별 (1, 0, -1) 평균내기

import pandas as pd
import re

file_path = 'sent_with_date_final.csv'
data = pd.read_csv(file_path)

# Define sentiment categories
positive = set(['감동/감탄', '기대감', '신기함/관심', '존경', '안심/신뢰', '환영/호의', '뿌듯함', 
                '고마움', '행복', '아껴주는', '기쁨', '흐뭇함(귀여움/예쁨)', '즐거움/신남', '편안/쾌적'])
negative = set(['안타까움/실망', '의심/불신', '절망', '불평/불만', '화남/분노', '경악', '공포/무서움', 
                '당황/난처', '부끄러움', '부담/안내킴', '서러움', '역겨움/징그러움', '우쭐댐/무시함', 
                '증오/혐오', '지긋지긋', '짜증', '패배/자기혐오', '한심함', '힘듦/지침', '슬픔', '불안/걱정', '귀찮음'])
neutral = set(['깨달음', '없음', '비장함', '놀람', '신기함/관심', '죄책감', '불쌍함/연민', '어이없음', '재미없음'])



def calculate_weighted_averages(sentiment_list):
    sentiments = re.findall(r"\['(.*?)', ([0-9.]+)\]", sentiment_list)
    if not sentiments:
        return 0, 0, 0  # Handle empty sentiment lists

    positive_scores = [float(score) for sentiment, score in sentiments if sentiment in positive]
    negative_scores = [-float(score) for sentiment, score in sentiments if sentiment in negative]
    neutral_scores = [float(score) for sentiment, score in sentiments if sentiment in neutral]

    total_scores = len(sentiments)

    pos_avg = sum(positive_scores) / total_scores if total_scores else 0
    neg_avg = sum(negative_scores) / total_scores if total_scores else 0
    neu_avg = sum(neutral_scores) / total_scores if total_scores else 0

    return pos_avg, neu_avg, neg_avg

# Apply the function and create new columns
data[['positive_avg', 'neutral_avg', 'negative_avg']] = data['sentiment'].dropna().apply(
    lambda x: pd.Series(calculate_weighted_averages(x))
)

# Save the processed data
weighted_output_path = 'sent_with_weighted_sentiment_averages2.csv'
data.to_csv(weighted_output_path, index=False)

In [ ]:
# 추세선 그리기

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import numpy as np

# Load data from a local CSV file
data = pd.read_csv('sent_with_weighted_sentiment_averages2.csv')

# Convert 'Date' to datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data.dropna(subset=['Date'])

# Ensure only numeric columns are aggregated
numeric_columns = ['positive_avg', 'neutral_avg', 'negative_avg']
grouped_data = data.groupby(['query', 'type', 'Date'])[numeric_columns].mean().reset_index()

# Correct negative_avg to be negative
grouped_data['negative_avg'] *= -1

# Filter only specified queries and types
queries_of_interest = ['강민경', '박나래', '설현', '아이린', '설민석', '홍진영', '양팡', '유희열', '조현아']
filtered_data = grouped_data[grouped_data['query'].isin(queries_of_interest)]

# Unique combinations of query and type
query_type_combinations = filtered_data[['query', 'type']].drop_duplicates()

# Define significant events by query
significant_events = {
    '아이린': {'2020-10-20': '논란 발생', '2020-10-22': '사과문'},
    '조현아': {'2014-12-05': '논란 발생', '2014-12-08': '사과문'},
    '박나래': {'2021-03-24': '논란 발생', '2021-03-25': '사과문', '2021-04-09': '활동 재개'},
    '설현': {'2016-05-03': '논란 발생', '2016-05-12': '사과문'},
    '설민석': {'2020-12-29': '논란 발생 및 사과문 기재'},
    '양팡': {'2020-08-05': '논란 발생', '2020-08-08': '사과문'},
    '유희열': {'2022-06-14': '논란 발생 및 1차 사과문 게재', '2022-07-18': '2차 사과문 게재'},
    '홍진영': {'2020-11-05': '논란 발생', '2020-11-06': '부인', '2020-12-18': '사과문'},
    '강민경': {'2020-07-15': '논란 발생', '2020-07-17': '사과문'}
}

# Set font for MacOS
plt.rcParams['font.family'] = 'AppleGothic'

# Create a PDF to save all plots
pdf_path_corrected = 'sentiment_trends_all_types.pdf'

# Re-plot graphs and save to PDF
def plot_each_row(pdf, max_points=30, cutoff_days=60):
    for _, row in query_type_combinations.iterrows():
        query, query_type = row['query'], row['type']
        subset = filtered_data[(filtered_data['query'] == query) & (filtered_data['type'] == query_type)]

        if not subset.empty:
            if query in significant_events:
                first_event_date = pd.to_datetime(list(significant_events[query].keys())[0])
                date_range_start = first_event_date - pd.DateOffset(days=cutoff_days)
                date_range_end = first_event_date + pd.DateOffset(days=cutoff_days)
                subset = subset[(subset['Date'] >= date_range_start) & (subset['Date'] <= date_range_end)]

            if len(subset) > max_points:
                subset = subset.sample(n=max_points, random_state=42).sort_values(by='Date')

            plt.figure(figsize=(12, 6))

            # Original data points
            plt.scatter(subset['Date'], subset['positive_avg'], label='긍정 평균', color='b', alpha=0.3)
            plt.scatter(subset['Date'], subset['neutral_avg'], label='중립 평균', color='g', alpha=0.3)
            plt.scatter(subset['Date'], subset['negative_avg'], label='부정 평균', color='r', alpha=0.3)

            # Rolling Mean (Smoothing)
            window_size = max(5, len(subset) // 10)
            for label, y_values, color in [
                ('긍정 추세', subset['positive_avg'], 'b'),
                ('중립 추세', subset['neutral_avg'], 'g'),
                ('부정 추세', subset['negative_avg'], 'r'),
            ]:
                rolling_avg = y_values.rolling(window=window_size, min_periods=1).mean()
                plt.plot(subset['Date'], rolling_avg, color=color, label=label)

            # Add significant event lines with labels
            if query in significant_events:
                colors = ['orange', 'purple', 'brown', 'red']
                for idx, (event_date, event_label) in enumerate(significant_events[query].items()):
                    event_date_parsed = pd.to_datetime(event_date)
                    plt.axvline(event_date_parsed, color=colors[idx % len(colors)], linestyle='--', linewidth=2, label=event_label)

            plt.title(f'{query} ({query_type}) 감정 추세 (Rolling Mean)')
            plt.xlabel('날짜')
            plt.ylabel('평균 감정 점수')
            plt.axhline(0, color='black', linestyle='--', alpha=0.7)
            plt.legend()
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.gcf().autofmt_xdate()
            pdf.savefig()
            plt.close()

# Save plots to the PDF file
def save_plots_to_pdf(file_path, plot_function):
    with matplotlib.backends.backend_pdf.PdfPages(file_path) as pdf:
        plot_function(pdf)

# Generate the corrected PDF
save_plots_to_pdf(pdf_path_corrected, plot_each_row)

print(f"PDF 파일이 생성되었습니다: {pdf_path_corrected}")


# 시기별 댓글 수 파악하기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf

# Load data from a local CSV file
file_path = 'sent_with_weighted_sentiment_averages2.csv'  # 파일 경로 수정
data = pd.read_csv(file_path)

# Convert 'Date' to datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data.dropna(subset=['Date'])

# Define queries of interest and significant events
queries_of_interest = ['강민경', '박나래', '설현', '아이린', '설민석', '홍진영', '양팡', '유희열', '조현아']
significant_events = {
    '아이린': {'2020-10-20': '논란 발생', '2020-10-22': '사과문'},
    '조현아': {'2014-12-05': '논란 발생', '2014-12-08': '사과문'},
    '박나래': {'2021-03-24': '논란 발생', '2021-03-25': '사과문', '2021-04-09': '활동 재개'},
    '설현': {'2016-05-03': '논란 발생', '2016-05-12': '사과문'},
    '설민석': {'2020-12-29': '논란 발생 및 사과문 기재'},
    '양팡': {'2020-08-05': '논란 발생', '2020-08-08': '사과문'},
    '유희열': {'2022-06-14': '논란 발생 및 1차 사과문 게재', '2022-07-18': '2차 사과문 게재'},
    '홍진영': {'2020-11-05': '논란 발생', '2020-11-06': '부인', '2020-12-18': '사과문'},
    '강민경': {'2020-07-15': '논란 발생', '2020-07-17': '사과문'}
}

# Filter data to only relevant queries and types
filtered_data = data[data['query'].isin(queries_of_interest)]

# Count comments by date for each query
comment_counts = filtered_data.groupby(['query', 'Date']).size().reset_index(name='count')

# Set font for MacOS
plt.rcParams['font.family'] = 'AppleGothic'

# Create a PDF to save all plots
pdf_path_counts = 'comment_counts_by_date_dark_green.pdf'

# Plot counts by date and save to PDF
def plot_comment_counts(pdf):
    for query in queries_of_interest:
        query_data = comment_counts[comment_counts['query'] == query]

        # Calculate x-axis range based on significant events
        if query in significant_events:
            first_event_date = pd.to_datetime(list(significant_events[query].keys())[0])
            last_event_date = pd.to_datetime(list(significant_events[query].keys())[-1])
            date_range_start = first_event_date - pd.DateOffset(months=1)
            date_range_end = last_event_date + pd.DateOffset(months=1)
            query_data = query_data[(query_data['Date'] >= date_range_start) & (query_data['Date'] <= date_range_end)]

        plt.figure(figsize=(12, 6))
        plt.plot(query_data['Date'], query_data['count'], marker='o', color='#228B22', label='댓글 수')

        # Add significant event lines with labels
        if query in significant_events:
            colors = ['orange', 'purple', 'brown', 'red']
            for idx, (event_date, event_label) in enumerate(significant_events[query].items()):
                event_date_parsed = pd.to_datetime(event_date)
                plt.axvline(event_date_parsed, color=colors[idx % len(colors)], linestyle='--', linewidth=2, label=event_label)

        plt.title(f'{query} 댓글 수 추이')
        plt.xlabel('날짜')
        plt.ylabel('댓글 수')
        plt.axhline(0, color='black', linestyle='--', alpha=0.7)
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.gcf().autofmt_xdate()
        pdf.savefig()
        plt.close()

# Save plots to the PDF file
def save_plots_to_pdf(file_path, plot_function):
    with matplotlib.backends.backend_pdf.PdfPages(file_path) as pdf:
        plot_function(pdf)

# Generate the corrected PDF
save_plots_to_pdf(pdf_path_counts, plot_comment_counts)

print(f"PDF 파일이 생성되었습니다: {pdf_path_counts}")


# 빈도분석 단순 count 표

In [20]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Load data from a local CSV file
file_path = 'frequency_analysis2.csv'  # 파일 경로 수정
data = pd.read_csv(file_path)

# Set font for MacOS
plt.rcParams['font.family'] = 'AppleGothic'

# Function to split 'count' column into word-frequency pairs
def split_count_column(row):
    try:
        return ast.literal_eval(row)  # Safely evaluate as a list of tuples
    except:
        return []

# Apply the splitting function to the 'count' column
data['count'] = data['count'].apply(split_count_column)

# Create a DataFrame to store query, key (시기), word, and frequency
query_word_list = []

for _, row in data.iterrows():
    query = row['query']
    key = row['key']  # 시기1, 시기2, 시기3
    word_freq_list = row['count']
    for word, freq in word_freq_list:
        query_word_list.append({'query': query, 'key': key, 'word': word, 'count': freq})

# Convert to DataFrame
word_counts = pd.DataFrame(query_word_list)

# Function to merge 시기1, 시기2, 시기3 into three columns sorted by descending count
def combine_period_columns(word_counts, query):
    tables = []
    for key in ['시기1', '시기2', '시기3']:
        period_data = word_counts[(word_counts['query'] == query) & (word_counts['key'] == key)]
        # Sort by descending count
        period_data = period_data.sort_values(by='count', ascending=False).reset_index(drop=True)
        tables.append(period_data.apply(lambda row: f"{row['word']}({row['count']})", axis=1).tolist())

    # Ensure all tables have the same number of rows by padding with empty values
    max_length = max(len(table) for table in tables)
    for table in tables:
        while len(table) < max_length:
            table.append('')  # Add empty strings for missing values

    # Combine tables into a single table with three columns
    combined_table = []
    for i in range(max_length):
        combined_table.append([tables[0][i], tables[1][i], tables[2][i]])
    return combined_table

# Create PDF path
pdf_file_path = 'query_word_frequencies_narrow_columns.pdf'

# Generate PDF
with PdfPages(pdf_file_path) as pdf:
    for query in word_counts['query'].unique():
        # Merge 시기1, 시기2, 시기3 into a single table
        combined_table = combine_period_columns(word_counts, query)

        # Define column headers
        col_labels = ['시기1', '시기2', '시기3']

        # Plot combined table
        fig, ax = plt.subplots(figsize=(8, min(12, 0.5 * len(combined_table) + 1)))  # Adjust height dynamically
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=combined_table, colLabels=col_labels, loc='center', cellLoc='center',
                         colWidths=[0.2, 0.2, 0.2])  # Narrower column widths

        # Set title
        ax.set_title(f'{query} 시기별 단어 빈도수', fontsize=14, fontweight='bold')

        # Adjust table formatting
        table.auto_set_font_size(False)
        table.set_fontsize(9)

        # Save page to PDF
        pdf.savefig(fig)
        plt.close(fig)

print(f"PDF 파일이 생성되었습니다: {pdf_file_path}")


/var/folders/m5/bfzh8qks2fx0rrw3lkq1q4s80000gn/T/ipykernel_58418/2488724768.py:84: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) AppleGothic.
  pdf.savefig(fig)
/var/folders/m5/bfzh8qks2fx0rrw3lkq1q4s80000gn/T/ipykernel_58418/2488724768.py:84: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) AppleGothic.
  pdf.savefig(fig)
/var/folders/m5/bfzh8qks2fx0rrw3lkq1q4s80000gn/T/ipykernel_58418/2488724768.py:84: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) AppleGothic.
  pdf.savefig(fig)
/var/folders/m5/bfzh8qks2fx0rrw3lkq1q4s80000gn/T/ipykernel_58418/2488724768.py:84: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) AppleGothic.
  pdf.savefig(fig)


PDF 파일이 생성되었습니다: query_word_frequencies_narrow_columns.pdf
